In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 1. Baca dataset Excel
df = pd.read_excel('StudentPerformanceFactors.xlsx')

In [ ]:
# 2. Tampilkan 5 baris awal untuk melihat struktur kolom
print(df.head())

In [ ]:
# 3. Deteksi kolom numerik kontinu
kolom_awal = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Kolom numerik kontinu:", kolom_awal)

# Imputasi

In [ ]:
df_imputed = df.copy()
num_cols = df_imputed.select_dtypes(include=[np.number]).columns.tolist() # Pilih kolom numerikal saja dan simpan dalam list
cat_cols = df_imputed.select_dtypes(exclude=[np.number]).columns.tolist() # Pilih kolom kategorikal saja dan simpan dalam list

# # Imputasi numerik dengan median
for c in num_cols:
#     print(f'Imputasi kolom {c}')
    med = df_imputed[c].median() # Hitung median  di kolom tersebut 
    # print(f'Median kolom {c}: {med}')
    df_imputed[c] = df_imputed[c].fillna(med) # Isi NA dengan median
# fillna() adalah fungsi di pandas untuk mengisi nilai yang kosong (NaN) dalam sebuah kolom atau DataFrame dengan nilai tertentu.



# # Imputasi kategorikal dengan modus (mode)
for c in cat_cols:
    mode_val = df_imputed[c].mode(dropna=True) 
    # Mencari modus, atau nilai yang paling sering muncul di data 
    # dropna=True untuk mengabaikan nilai NaN saat mencari modus
    # print(f'Modus kolom {c}: {mode_val.iloc[0] if not mode_val.empty else "Tidak ada modus"}')
    if not mode_val.empty: # Cek apakah modus ditemukan
        df_imputed[c] = df_imputed[c].fillna(mode_val.iloc[0]) # Isi NA dengan modus iloc[0] adalah cara untuk mengakses nilai pertama dari hasil modus
print('Sisa NA setelah imputasi:')
print(df_imputed.isna().sum())


In [ ]:
# 4. Fungsi kategorisasi otomatis berdasarkan interval (tanpa menambah kolom baru)
def kategorisasi_otomatis(df_imputed, kolom, bins, labels):
    # ✅ Validasi jumlah bins dan label
    if len(bins) - 1 != len(labels):
        raise ValueError(f"❌ Jumlah bins dan labels tidak sesuai pada kolom '{kolom}'")

    # ✅ Pastikan label unik
    if len(set(labels)) != len(labels):
        raise ValueError(f"❌ Label duplikat terdeteksi pada kolom '{kolom}': {labels}")

    # ✅ Proses kategorisasi langsung menggantikan kolom lama
    df_imputed[kolom] = pd.cut(
        df_imputed[kolom],
        bins=bins,
        labels=labels,
        include_lowest=True,
        ordered=True  # urutan kategori bermakna
    )
    return df_imputed

In [ ]:
# 5. Daftar kolom dan aturan kategorisasi
kolom_kategori = {
    'Hours_Studied': ([0, 10, 20, 30], ['Low', 'Medium', 'High']),
    'Attendance': ([0, 74, 100], ['Not Met', 'Met']),
    'Sleep_Hours': ([0, 4, 6, 8, 10], ['Low', 'Medium', 'High', 'Extreme']),
    'Previous_Scores': ([0, 40, 54, 62, 70, 78, 86, 100], ['E', 'D', 'C', 'BC', 'B', 'AB', 'A']),
    'Tutoring_Sessions': ([0, 2, 4, 6, 8], ['Low', 'Medium', 'High', 'Extreme']),
    'Physical_Activity': ([0, 2, 4, 6], ['Low', 'Medium', 'High']),
    'Exam_Score': ([0, 40, 54, 62, 70, 78, 86, 100], ['E', 'D', 'C', 'BC', 'B', 'AB', 'A'])
}

# 6. Terapkan kategorisasi ke semua kolom sesuai aturan
for kolom, (bins, labels) in kolom_kategori.items():
    if kolom in df_imputed.columns:
        df_imputed = kategorisasi_otomatis(df_imputed, kolom, bins, labels)
        print(f"✔️ Kolom '{kolom}' berhasil dikategorikan tanpa menambah kolom baru")
        
# 7. Pastikan urutan kolom tetap seperti kolom_awal
df_imputed = df_imputed[[kol for kol in kolom_awal if kol in df_imputed.columns]]

print("\n✅ Urutan kolom akhir sudah disesuaikan kembali.")

# 6. Simpan hasil ke file baru
output_path = 'data_kategorikal.xlsx'
df.to_excel(output_path, index=False)
print(f"Data berhasil disimpan ke {output_path}")

In [ ]:

# ✅ Note: Simpan hasil dataset ke file Excel/CSV
df_imputed.to_excel("hasil_dataset_kategorisasi.xlsx", index=False)
print("\n📁 Dataset hasil kategorisasi telah disimpan ke 'hasil_dataset_kategorisasi.xlsx'")